In [1]:
# from bs4 import BeautifulSoup

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import pandas as pd
import numpy  as np 
import re
import datetime
import os,glob
import json
from json.decoder import JSONDecodeError

In [2]:
# preparing list of existing(previously created by scrapping) pandas linkedin .h5 files
search_dir = "/home/lubuntu/scrap/glass/" #directory where from where h5 is read s
def list_per_time(search_dir=search_dir):
    files = [x for x in (filter(os.path.isfile, glob.glob(search_dir + "glassdoor*.h5"))) ] 
    files_sorted= sorted(files, key=os.path.getmtime)
    print(files_sorted[:10])
    return files_sorted

In [3]:
# start web address 
start_url= 'https://www.glassdoor.com/Job/berlin-biotechnology-jobs-SRCH_IL.0,6_IC2622109_KE7,20.htm?fromAge=3'

In [4]:
# makes the source tree format like

def beautify(url):
    hdr = {'User-Agent': 'Marozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)

    return BeautifulSoup(page, "html.parser")

In [5]:
jobs = beautify(start_url)
#retrieving information how many pages with jobs ads are
def how_many_pages(start_url):
    how_many_pages_job= jobs.findAll('div', {'class' : "cell middle hideMob padVertSm"})
    for x, result in enumerate(how_many_pages_job):
        print('last page is --->', result.text[-2:])
    return result.text[-2:]

In [6]:
pages=how_many_pages(start_url)
pages

last page is --->  1


' 1'

In [7]:
start_urls=[]# based on the number of summary webpages we generate their names in the table to later check
def gen_table_urls(start_url, pages ): #based on number of pages with links function should generate web addres for every page
    pp='&&p='
    for i in range (int(pages)):
#         print(i+1,'-->', start_url[:-14]+'_IP'+str(i+1)+start_url[-14:])
        start_urls.append(start_url[:-14]+'_IP'+str(i+1)+start_url[-14:])
        

In [8]:
gen_table_urls(start_url, pages ) #table with wabpages to check is generated

In [9]:
start_urls

['https://www.glassdoor.com/Job/berlin-biotechnology-jobs-SRCH_IL.0,6_IC2622109_KE7,20_IP1.htm?fromAge=3']

In [10]:
res = [] #table with all relevant links to jobs
for x, job in enumerate(start_urls):
    job = beautify(start_urls[x])
    j_links=job.findAll('a', attrs={'href': re.compile("^https://www.glassdoor.com/partner/jobListing.htm?")}) #filtering links to webpages of selected jobs
    jobs_links = [link.get('href') for link in j_links] #creating a list of job links
    for i in jobs_links: 
        if i not in res:
            res.append(i)


In [11]:
print('There are --> ',len(res), 'jobs and links')

There are -->  7 jobs and links


In [12]:
#patterns to remove from description
clean_patterns= ['&lt;', '/&gt', ';br',';li',';/ul','li&gt;', 'h2&gt;', 'ul&gt;', 'br ;', '/', 'div&gt;', 'h3&gt;', 'h4&gt;' ] 

In [13]:
description='XXXXXXX' #when there is problem with reading field with job description to be able go further fill with XXXXXX

In [14]:
def clean_job_descri(description=description, clean_patterns=clean_patterns): # function which is cleaning job description from tags defined before
    
    for x,clean in enumerate(clean_patterns):
#         print(x)
        pattern=re.compile(clean_patterns[x])
        description= pattern.sub(" ", description)
    pattern=re.compile(r"strong&gt;")
    description= pattern.sub("\n\n *---*", description)
#     print(description)
    return description

In [15]:
    def create_empty_record(job_dict): #when there is a problem with urls fill record with default data so the scrapping is going further
                      
        description=job_dict
#         job_descr=clean_job_descri(description)
        job_name = 'XXXXXXX'
        company_name = 'XXXXXXX'
        job_descr=job_name
        timestamp = datetime.datetime.now()
        datePosted = '2020-07-01'
        validThrough = '2020-07-01'
        job_url = 'url'
        place_name='addressLocality'
        place_name_region = 'addressRegion'
        latitude = '52.5177'
        longitude = '13.4055'
        employmentType = 'UNKNOWN'
        return job_name, company_name, job_descr, timestamp, datePosted, validThrough, job_url, place_name, place_name_region, latitude, longitude, employmentType

In [16]:
    #collecting information from dictionary like structure
    def create_record(job_dict):
                      
        description=job_dict['description']
        clean_job_descri(description)  
        job_name = job_dict['title']
        company_name = job_dict['hiringOrganization']['name']
        job_descr=clean_job_descri(description)
        timestamp = datetime.datetime.now()
        datePosted = job_dict['datePosted']
        validThrough = job_dict['validThrough']
        job_url = job_dict['url']
        place_name=job_dict['jobLocation']['address']['addressLocality']
        place_name_region = job_dict['jobLocation']['address']['addressRegion']
        latitude = job_dict['jobLocation']['geo']['latitude']
        longitude = job_dict['jobLocation']['geo']['longitude']
        employmentType = job_dict['employmentType']
        return job_name, company_name, job_descr, timestamp, datePosted, validThrough, job_url, place_name, place_name_region, latitude, longitude, employmentType

In [17]:
def get_job_record(url): #retrieving information from webpages if there are some json errors on url  using create_emty_record_function
    j = beautify(url)
    try:
        job_dict=json.loads(r"".join(j.find("script", {"type":"application/ld+json"}).contents), strict=False) #there is there job description location etc
        job_name, company_name, job_descr, timestamp, datePosted, validThrough, job_url, place_name, place_name_region, latitude, longitude, employmentType=create_record(job_dict)
    except JSONDecodeError as e:
        print('there was some json error with url:', url)
        job_dict=j.find("script", {"type":"application/ld+json"}).contents
        job_name, company_name, job_descr, timestamp, datePosted, validThrough, job_url, place_name, place_name_region, latitude, longitude, employmentType=create_empty_record(job_dict)

        

        
    return job_name, company_name, job_descr, timestamp, datePosted, validThrough, job_url, place_name, place_name_region, latitude, longitude, employmentType
        
        

In [18]:
# =================================temporary  tables to store  info from webpages
job=[] #ok
company=[] #ok
place=[] #ok
job_d=[] #ok
time_s=[] #ok
datePosted_t=[] #ok
validThrough_t=[] #ok
job_url_t=[] #ok
place_name_region_t =[]#ok
latitude_t =[]#ok
longitude_t =[]#ok
employmentType_t =[]

In [19]:
# populating tables above with data from particular jobs
def populate_temp_tables(url_list):
       
    for x,url in enumerate(url_list):
        job_name, company_name, job_descr, timestamp, datePosted, validThrough, job_url, place_name, place_name_region, latitude, longitude, employmentType = get_job_record(url)
        print(x+1,'-->',url)
        job.append(job_name)
        company.append(company_name)        
        job_d.append(job_descr)
        time_s.append(timestamp)
        datePosted_t.append(datePosted)
        validThrough_t.append(validThrough)
        job_url_t.append(job_url)
        place.append(place_name)
        place_name_region_t.append(place_name_region)
        latitude_t.append(latitude)
        longitude_t.append(longitude)
        
    return job, company, place, job_d, time_s, datePosted_t, validThrough_t, job_url_t, place_name_region_t, latitude_t, longitude_t, employmentType_t
 

==========================================================================================

In [30]:
populate_temp_tables(res)

1 --> https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=4120&s=58&guid=0000017399f6347292143ecce622b619&src=GD_JOB_AD&t=SR&extid=1&exst=EL&ist=&ast=EL&slr=true&cs=1_254241d6&cb=1596015916299&jobListingId=3636213042
2 --> https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=474845&s=58&guid=0000017399f6347292143ecce622b619&src=GD_JOB_AD&t=SR&extid=1&exst=EL&ist=&ast=EL&slr=true&cs=1_1a0a2e08&cb=1596015916299&jobListingId=3635357570
3 --> https://www.glassdoor.com/partner/jobListing.htm?pos=103&ao=1034696&s=58&guid=0000017399f6347292143ecce622b619&src=GD_JOB_AD&t=SR&extid=1&exst=EL&ist=&ast=EL&slr=true&cs=1_118504de&cb=1596015916299&jobListingId=3548761211
4 --> https://www.glassdoor.com/partner/jobListing.htm?pos=104&ao=437149&s=58&guid=0000017399f6347292143ecce622b619&src=GD_JOB_AD&t=SR&extid=1&exst=EL&ist=&ast=EL&slr=true&cs=1_8a473e24&cb=1596015916299&jobListingId=3589386064
5 --> https://www.glassdoor.com/partner/jobListing.htm?pos=105&ao=437149&s=58&guid=0000017399

(['DM.132.20b Research Associate (Postdoc) - Chronic and genetic kidney disease',
  'Senior Specialist, Apheresis Audit - Germany (location Flexible)',
  'Head of External Activation and Identity Management - Director',
  'FRONT END DEVELOPER (M/F/D)',
  'Senior Manager Health Economics & Market Access (m/f/d)',
  'Laboratory technician (m/f/d)',
  'Senior scientist (m/f/d)'],
 ['Charité',
  'Bristol-Myers Squibb',
  'Bayer',
  'Clincase',
  'BIOTRONIK',
  'PERMACON GmbH Berlin',
  'PERMACON GmbH Berlin'],
 ['Berlin', 'Berlin', 'Berlin', 'Berlin', 'Berlin', 'Berlin', 'Berlin'],
 ['  Unternehmensbeschreibung       Die Charit&amp;eacute; &amp;ndash; Universit&amp;auml;tsmedizin Berlin ist eine gemeinsame Einrichtung der Freien Universit&amp;auml;t Berlin und der Humboldt-Universit&amp;auml;t zu Berlin. Sie hat als eines der gr&amp;ouml;&amp;szlig;ten Universit&amp;auml;tsklinika Europas mit bedeutender Geschichte eine f&amp;uuml;hrende Rolle in Forschung, Lehre und Krankenversorgung inne

In [31]:
# ====================creating pandas dataframe with all the temp tables populated before

In [32]:
jobs_glassdoor = pd.DataFrame({'job_name' : job,
                               'company_name' : company,
                               'job_descr' : job_d,
                               'timestamp': time_s,
                               'datePosted': datePosted_t,
                               'validThrough' : validThrough_t,
                               'job_url' : job_url_t,
                               'place_name' : place,
                               'place_name_region' : place_name_region_t,
                               'latitude_' : latitude_t,
                               'longitude' : longitude_t})

In [33]:
jobs_glassdoor.info() #double checking 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   job_name           7 non-null      object        
 1   company_name       7 non-null      object        
 2   job_descr          7 non-null      object        
 3   timestamp          7 non-null      datetime64[ns]
 4   datePosted         7 non-null      object        
 5   validThrough       7 non-null      object        
 6   job_url            7 non-null      object        
 7   place_name         7 non-null      object        
 8   place_name_region  7 non-null      object        
 9   latitude_          7 non-null      object        
 10  longitude          7 non-null      object        
dtypes: datetime64[ns](1), object(10)
memory usage: 744.0+ bytes


In [34]:
# generating file name with timestamp info for later comparison
def gen_h5_file_with_timestamp(name='name'):
    timestamp = datetime.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".h5"
    return file_name


In [35]:
file_name= gen_h5_file_with_timestamp(name='glassdoor-')
file_name

'glassdoor-Wednesday29July202007-11-46-53.h5'

In [36]:
# saving dataframe in .h5 file for later use
jobs_glassdoor.to_hdf('./glass/'+file_name, key='df', mode='w') 

In [37]:
# loading last( creation time) two dfs for comparison
def read_two_last_dfs():
    files_sorted=list_per_time()
    f1=files_sorted[-2:-1]
    f2=files_sorted[-1:]
    df1=pd.read_hdf(f1[0], 'df')
    df2=pd.read_hdf(f2[0], 'df')
    return df1, df2

In [38]:
df1, df2=read_two_last_dfs()

['/home/lubuntu/scrap/glass/glassdoor-Wednesday29July202007-11-45-38.h5', '/home/lubuntu/scrap/glass/glassdoor-Wednesday29July202007-11-46-53.h5']


In [39]:
# overview of loaded dfs
df1.info(), df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_name           0 non-null      float64
 1   company_name       0 non-null      float64
 2   job_descr          0 non-null      float64
 3   timestamp          0 non-null      float64
 4   datePosted         0 non-null      float64
 5   validThrough       0 non-null      float64
 6   job_url            0 non-null      float64
 7   place_name         0 non-null      float64
 8   place_name_region  0 non-null      float64
 9   latitude_          0 non-null      float64
 10  longitude          0 non-null      float64
dtypes: float64(11)
memory usage: 0.0 bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   job_name           7 no

(None, None)

In [40]:
# checking what new/diffrent records(False) are new in df2
df2['job_url'].isin(df1['job_url'])

0    False
1    False
2    False
3    False
4    False
5    False
6    False
Name: job_url, dtype: bool

In [41]:
# checking what records are new in df2 compared to df1(~not in) as overview
df2.loc[~df2['job_url'].isin(df1['job_url'])]

,job_name,company_name,job_descr,timestamp,datePosted,validThrough,job_url,place_name,place_name_region,latitude_,longitude
0,DM.132.20b Research Associate (Postdoc) - Chro...,Charité,Unternehmensbeschreibung Die Charit&am...,2020-07-29 11:46:26.861771,2020-07-29,2020-08-28,https://www.glassdoor.de/job-listing/dm-132-20...,Berlin,16,52.5177,13.4055
1,"Senior Specialist, Apheresis Audit - Germany (...",Bristol-Myers Squibb,"At Bristol Myers Squibb, we are inspired by a ...",2020-07-29 11:46:29.257026,2020-07-28,2020-08-28,https://www.glassdoor.de/job-listing/senior-sp...,Berlin,16,52.5177,13.4055
2,Head of External Activation and Identity Manag...,Bayer,\n\n *---*Bayer is a global enterprise with c...,2020-07-29 11:46:31.373281,2020-07-29,2020-08-28,https://www.glassdoor.de/job-listing/head-of-e...,Berlin,16,52.5177,13.4055
3,FRONT END DEVELOPER (M/F/D),Clincase,Clincase is currently seeking a Front End Deve...,2020-07-29 11:46:33.890353,2020-07-28,2020-08-28,https://www.glassdoor.de/job-listing/front-end...,Berlin,16,52.5177,13.4055
4,Senior Manager Health Economics & Market Acces...,BIOTRONIK,BIOTRONIK is one of the leading manufacturers ...,2020-07-29 11:46:36.100561,2020-07-29,2020-08-28,https://www.glassdoor.de/job-listing/senior-ma...,Berlin,16,52.5177,13.4055
5,Laboratory technician (m/f/d),PERMACON GmbH Berlin,We are hiring\n \nAt PERMACON you will find...,2020-07-29 11:46:38.113918,2020-07-28,2020-08-28,https://www.glassdoor.de/job-listing/laborator...,Berlin,16,52.5177,13.4055
6,Senior scientist (m/f/d),PERMACON GmbH Berlin,We are hiring\n \nAt PERMACON you will find...,2020-07-29 11:46:41.419087,2020-07-28,2020-08-28,https://www.glassdoor.de/job-listing/senior-sc...,Berlin,16,52.5177,13.4055


In [42]:
print('=============links to new jobs since last checking=============\n')
for x,item in enumerate(df2.job_url.values[~df2['job_url'].isin(df1['job_url'])]):
    print(x, '-->',item )

=============links to new jobs since last checking=============

0 --> https://www.glassdoor.de/job-listing/dm-132-20b-research-associate-postdoc-chronic-and-genetic-kidney-disease-charité-JV_IC2622109_KO0,72_KE73,80.htm?jl=3636213042
1 --> https://www.glassdoor.de/job-listing/senior-specialist-apheresis-audit-germany-location-flexible-bristol-myers-squibb-JV_IC2622109_KO0,59_KE60,80.htm?jl=3635357570
2 --> https://www.glassdoor.de/job-listing/head-of-external-activation-and-identity-management-director-bayer-JV_IC2622109_KO0,60_KE61,66.htm?jl=3548761211
3 --> https://www.glassdoor.de/job-listing/front-end-developer-mfd-clincase-JV_IC2622109_KO0,23_KE24,32.htm?jl=3589386064
4 --> https://www.glassdoor.de/job-listing/senior-manager-health-economics-market-access-mfd-biotronik-JV_IC2622109_KO0,49_KE50,59.htm?jl=3612941022
5 --> https://www.glassdoor.de/job-listing/laboratory-technician-mfd-permacon-gmbh-berlin-JV_IC2622109_KO0,25_KE26,46.htm?jl=3635236464
6 --> https://www.glassdoor.de/j

In [46]:
# generating file name with timestamp info for later comparison
def gen_md_file_with_timestamp(name='name'):
    timestamp = datetime.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".md"
    return file_name

In [47]:
md_file_name= gen_md_file_with_timestamp(name='selected-bio-')
md_file_name

'selected-bio-Wednesday29July202007-11-49-51.md'

====list of the interesting columns which should be displayed processes

In [48]:
l3= list(df2.job_name.values+'    \n'+df2.company_name.values+ '    \n'+df2.job_url.values+'   '+df2.job_descr.values)

In [49]:
timestamp = datetime.datetime.now()
ts1 = timestamp.strftime("%Y-%m-%d")

def df_date_filter(_date= ts1): # you could give date in a format '2020-07-21'
    

    l4= list(df2.job_name.values[df2.datePosted==ts1]+
             '    \n'+df2.company_name.values[df2.datePosted==ts1]+
             '    \n'+df2.job_url.values[df2.datePosted==ts1]+
             '    \n'+df2.job_descr.values[df2.datePosted==ts1])
    return l4


In [52]:
def save_Table_to_file(temp_file=l3): #saving content to md file
    
    with open('./glass/'+md_file_name, 'a') as f:
        for x, item in enumerate(temp_file):
            temp= '\n'+'> ##  Job nr:'+str(x+1)+ '/'+str(len(temp_file))+'------------------------------   \n'
            temp1='    \n'
            f.write(temp)
            f.write(md_file_name) 
            f.write(temp1)
            f.write("%s\n" % item)

========Saving subset of jobs from today

In [53]:
save_Table_to_file(df_date_filter(_date= ts1)) # you could give date in a format '2020-07-21'

========Saving subset of jobs from particular day


In [54]:
save_Table_to_file(df_date_filter(_date= '2020-07-29')) 

====There is possibility to define a uiversal filter between df1 and df2 ie between the present and last df


In [55]:
filter1= ~df2['job_url'].isin(df1['job_url'])

filter2= df2.datePosted=='2020-07-29'

In [56]:

def df_difference_filter(filter=filter1): # you could give date in a format '2020-07-21'
    

    l4= list(df2.job_name.values[filter]+
             '    \n'+df2.company_name.values[filter]+
             '    \n'+df2.job_url.values[filter]+
             '    \n'+df2.job_descr.values[filter])
    return l4


In [57]:
# generate file with diffrence to last 
md_file_name= gen_md_file_with_timestamp(name='selected-ml-diffrerence-')
md_file_name
save_Table_to_file(df_difference_filter(filter1)) 